## Loading data

In [46]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-01 22:54:52--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.184.207, 108.177.121.207, 209.85.145.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.184.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   110MB/s    in 0.6s    

2024-11-01 22:54:52 (110 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [54]:
import PIL
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

x_train = []
y_train = []


cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")
i = 0
NUMBER_OF_EXAMPLES = 1000
while i < NUMBER_OF_EXAMPLES:
  if i % 2 == 0:
    im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")
    im_resized = im.resize((300, 300))
    x_train.append(np.array(im_resized))
    y_train.append(1)
  else:
    im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")
    im_resized = im.resize((300, 300))
    x_train.append(np.array(im_resized))
    y_train.append(0)
  i += 1

dogs = sorted(os.listdir(dogs_dir))
cats = sorted(os.listdir(cats_dir))



## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [55]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = preprocess_input(x_train)


pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(300, 300, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)
model = Sequential()
model.add(pretrained_model)
model.add(Dense(1, activation='sigmoid', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05)))


In [56]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [57]:
model.fit(
    x_train,
    y_train,
    batch_size=16,
    epochs=1
)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1464s 22s/step - accuracy: 0.6858 - loss: 1.0900


In [ ]:
prediction = model.predict(x_train)
print(x_train)
right = 0
total = len(prediction)
for i, pred in enumerate(prediction):
    if pred[0] > 0.5:
      predicted = 'Cat'
    else:
      predicted = 'Dog'
    if y_train[i] == 1:
      actual = 'Cat'
    else:
      actual = 'Dog'
    if predicted == actual:
      right+=1
    print("Predicted:", {predicted}, "Actual:", {actual})
print(right/total)

 6/32 ━━━━━━━━━━━━━━━━━━━━ 4:28 10s/step